<h1>Building and Training Siamese Neural Network</h1>

<h3>Data loading and preprocessing</h3>

<h3>Model building and training</h3>

In [1]:
import torch
from torchsummary import summary
from src.model.FaceNet import SiameseNetwork

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [3]:
random_input_example = torch.randn(1, 3, 512, 512, device=device)
model = SiameseNetwork()

In [4]:
model.to(device)
print(f'Model is on cuda: {next(model.parameters()).is_cuda}')

Model is on cuda: True


In [5]:
next(model.parameters()).is_cuda

True

In [6]:
summary(model, input_size=(3, 512, 512))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1        [-1, 128, 510, 510]           3,584
         MaxPool2d-2        [-1, 128, 255, 255]               0
             PReLU-3        [-1, 128, 255, 255]             128
            Conv2d-4         [-1, 64, 253, 253]          73,792
         MaxPool2d-5         [-1, 64, 126, 126]               0
             PReLU-6         [-1, 64, 126, 126]              64
            Conv2d-7         [-1, 32, 124, 124]          18,464
         MaxPool2d-8           [-1, 32, 62, 62]               0
             PReLU-9           [-1, 32, 62, 62]              32
           Conv2d-10           [-1, 16, 60, 60]           4,624
        MaxPool2d-11           [-1, 16, 30, 30]               0
            PReLU-12           [-1, 16, 30, 30]              16
           Conv2d-13            [-1, 8, 28, 28]           1,160
        MaxPool2d-14            [-1, 8,

In [7]:
y = model(random_input_example)
print(f'Output shape: {y.shape}')

Output shape: torch.Size([1, 256])


<h3>Model evaluation</h3>